<a href="https://colab.research.google.com/github/josephmhernandez/SkinCancerDetection/blob/master/SkinCancer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exploring Ham dataset


Exploring the HAM10000 data set. 
This dataset contains images of skin lesions that have been classified into 7 different categories: 

1.  bcc -> Basal cell carcinoma: common variant of epithelial skin cancer
2. bkl -> Benign keratosis - Generic class. Dermatoscopically, there are three subgroups that could look different but in this dataset they are all grouped together because they are similar biologically and reported under the same generic term. Hard to distinguish between this and melanoma. 
3. df -> Dermatofibroma: benign skin lesion
4. nv -> Melanocytic nevi are benign neoplasms of melanocytes and appear variants that can differ significantly from a dermatoscopic view. 
5. mel -> Melanoma: malignant neoplasm that may appear in a variety of variants. This dataset excludes non-pigmented subungual, ocular or mucosal melanoma.
6. vasc -> Vascular skin lesions could be benign or not.  
7. akiec -> Actinic Keratoses and Intraepithelial Carcinoma are common non-invasive, variants of squamous cell carcinoma that can be treated locally without surgery.

In [0]:
#Mount Google Drive. 
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


## Value Counts


In [31]:
#Open the csv file and output diagnostics for dataset. 
import pandas as pd
#Import csv file. 
metaDataCsv = pd.read_csv('gdrive/My Drive/Skin Cancer Detection/HAM10000_metadata.csv')

cols = metaDataCsv.columns
_temp_str = 'Columns: '
for c in cols:
    _temp_str += c + ', '
_temp_str = _temp_str[:len(_temp_str)-2]
print(_temp_str)

for c in cols:
    print()
    print(c + ':')
    print(metaDataCsv[c].value_counts())

Columns: lesion_id, image_id, dx, dx_type, age, sex, localization

lesion_id:
HAM_0001863    6
HAM_0000835    6
HAM_0005263    6
HAM_0003789    6
HAM_0002364    5
              ..
HAM_0002118    1
HAM_0000408    1
HAM_0006690    1
HAM_0004152    1
HAM_0004374    1
Name: lesion_id, Length: 7470, dtype: int64

image_id:
ISIC_0032470    1
ISIC_0031829    1
ISIC_0030691    1
ISIC_0031579    1
ISIC_0028900    1
               ..
ISIC_0031507    1
ISIC_0033737    1
ISIC_0024714    1
ISIC_0024856    1
ISIC_0032968    1
Name: image_id, Length: 10015, dtype: int64

dx:
nv       6705
mel      1113
bkl      1099
bcc       514
akiec     327
vasc      142
df        115
Name: dx, dtype: int64

dx_type:
histo        5340
follow_up    3704
consensus     902
confocal       69
Name: dx_type, dtype: int64

age:
45.0    1299
50.0    1187
55.0    1009
40.0     985
60.0     803
70.0     756
35.0     753
65.0     731
75.0     618
30.0     464
80.0     404
85.0     290
25.0     247
20.0     169
5.0       86
1

## Distrubtion of Images

In [30]:
#Distribution of diagnosed leisons. 
dx_counts = metaDataCsv['dx'].value_counts()

_size = metaDataCsv['dx'].size
print('dx:\tCount:\tPercent:')
for key,val in dx_counts.iteritems():
    _percentage = val / _size * 100
    print(key + '\t'+ str(val) + '\t' + '{0:.2f}'.format(_percentage) + '%')
print('Total images: ' + str(_size))
    

dx:	Count:	Percent:
nv	6705	66.95%
mel	1113	11.11%
bkl	1099	10.97%
bcc	514	5.13%
akiec	327	3.27%
vasc	142	1.42%
df	115	1.15%
Total images: 10015


From looking at the data we can see the type of skin leison in this dataset is not evenly distributed througout the seven types. We can also see that there were different pictures from the same leison. When differentiating images we should use the image_id as a key. From looking at the percentage of the vlaues are in nv the baseline for the model will need to be an accuracy above 67%.

For image classification, the only necessary data from this csv file will be the image_id and the diagnosis. The images were named with the image_id. 

Training data for the model will be the pixels of the image and the diagnosis (dx) will be label. 



# Preprocessing Images


In [0]:
#Mount to google drive. 
#Create dictionary with image_id, dx for the training labels. 
#Open all images in the google drive folders and convert images to a 3 dimensional
#   tesnor (height, width, depth). 
#Save image tesnors as .npy files so that it'll be quicker to train the model. 
    #Storing images as .npy files will allow us to quickly access the pixel 
        #values for each image in order to train the model quicker. This format 
        #will also allow us to create a funciton generator to use multiprocessing
        #to train/test the neural network. 